In [1]:
from google.colab import drive
drive.mount('/content/gdrive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
%cd /content/gdrive/My Drive/EVA/

/content/gdrive/My Drive/EVA


In [3]:
!pwd

/content/gdrive/My Drive/EVA


In [4]:
!ls -l

ls: bg_scld_opmzd: No such file or directory
ls: data: No such file or directory
ls: 'EVA 15A Dataset': No such file or directory
total 6429681
-rw------- 1 root root  944366544 May  9 11:01  bg_fg_images_masks_str.zip
-r-------- 1 root root 5574672534 May  9 09:14  bg_fg_images_str.zip
-rw------- 1 root root    1532043 May  9 11:36  bg_images.zip
lrw------- 1 root root          0 May  9 11:00  bg_scld_opmzd -> /content/gdrive/.shortcut-targets-by-id/1Fi2K2QTpKgfmMd0dH9xFptBlNEqyF0uY/bg_scld_opmzd
lrw------- 1 root root          0 May 10 08:51  data -> /content/gdrive/.shortcut-targets-by-id/1uAsyblbBx3APE6TkcgwVbXvdeybQXge9/data
-rw------- 1 root root   61541083 May  9 08:31  depth-mask.zip
lrw------- 1 root root          0 May 10 08:52 'EVA 15A Dataset' -> /content/gdrive/.shortcut-targets-by-id/1uAsyblbBx3APE6TkcgwVbXvdeybQXge9/data
-rw------- 1 root root    1871572 May  9 11:56  fg_images.zip
drwx------ 2 root root       4096 May  6 08:00  Folder1
drwx------ 2 root root       4096 

In [5]:
from zipfile import ZipFile
import zipfile
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import io
from itertools import groupby
import cv2
from tqdm.auto import tqdm
from pathlib import Path
from time import time
from torchvision import datasets
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.utils.data import random_split
from PIL import Image
import torchvision.transforms as T
import os
import torch
sns.set()

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Let's see if the dataset works

In [0]:
fg_bg_zip =  ZipFile('bg_fg_images_str.zip', 'r')

In [0]:
all_files = [info.filename for info in fg_bg_zip.infolist() if not info.is_dir()]

In [0]:
len(all_files)

428000

In [0]:
fg_bg_zip =  ZipFile('bg_images.zip', 'r')

In [0]:
all_files = [info.filename for info in fg_bg_zip.infolist() if not info.is_dir()]
len(all_files)

107

In [0]:
for idx, file in enumerate(all_files):
    imgdata = fg_bg_zip.read(file)
    img = Image.open(io.BytesIO(imgdata))
    img = img.convert("RGB")

    if idx % 1000 == 0:
        print(f'processed {file} to {img.size}')

In [0]:
class DenseDepth(Dataset):
    '''
    DenseDepth Dataset

    Input is fg_bg image AND bg image
    Target is fg_bg_mask AND depth_fg_bg image
    '''
    def __init__(self, root, data='fg_bg', train=True, transform=None, target_transform=None, download=False):
        self.root = root
        self.transform = transform
        self.target_transform = target_transform

        # open the respective data and target
        bg_zip = ZipFile(os.path.join(self.root,'bg_images.zip'), 'r') # bg
        fg_bg_zip =  ZipFile(os.path.join(self.root,'bg_fg_images_str.zip'), 'r') # fg_bg
        fg_bg_mask_zip = ZipFile(os.path.join(self.root,'bg_fg_images_masks_str.zip'), 'r') # fg_bg_mask
        depth_fg_bg_zip =  ZipFile(os.path.join(self.root,'EVA 15A Dataset/depth_output/bgScaled_0.zip'), 'r') # target

        bg_paths = [info.filename for info in bg_zip.infolist() if not info.is_dir()]
        fg_bg_paths = [info.filename for info in fg_bg_zip.infolist() if not info.is_dir()]
        fg_bg_mask_paths = [info.filename for info in fg_bg_mask_zip.infolist() if not info.is_dir()]
        depth_fg_bg_paths = [info.filename for info in depth_fg_bg_zip.infolist() if not info.is_dir()]


        # fg_bg_masks_w_depth = zip(fg_bg_paths, fg_bg_mask_paths)


        assert(len(bg_paths) == 107)
        assert(len(fg_bg_paths) == 428000)
        assert(len(fg_bg_mask_paths) == 428000)
        #assert(len(depth_fg_bg_paths) == 400000)

        #print(f'found {len(bg_paths)} bg images, {len(fg_bg_paths)} fg_bg images, {len(fg_bg_mask_paths)} fg_bg_mask images, {len(depth_fg_bg_paths)} depth_fg_bg images')


        if data == 'fg_bg':
            self.image_zip = fg_bg_zip
            self.image_paths = fg_bg_paths
        elif data == 'bg':
            self.image_zip = bg_zip
            self.image_paths = bg_paths
        elif data == 'fg_bg_mask':
            self.image_zip = fg_bg_mask_zip
            self.image_paths = fg_bg_mask_paths
        elif data == 'depth_fg_bg':
            self.image_zip = depth_fg_bg_zip
            self.image_paths = depth_fg_bg_paths
        else:
            raise f'{data} is not a valid option'

        self.targets = []

        # train_size = int(0.8 * len(self.image_paths))
        # test_size = len(self.image_paths) - train_size

        # train_paths, test_paths = torch.utils.data.random_split(, [train_size, test_size])

        # if train:

    def __getitem__(self, index):
        imgdata = self.image_zip.read(self.image_paths[index])
        img = Image.open(io.BytesIO(imgdata))
        img = img.convert("RGB")
        img = np.array(img)

        if self.transform is not None:
            img = self.transform(img)
        
        return img

        # filepath = self.image_paths[index]
        # img = Image.open(filepath)
        # img = img.convert("RGB")
        # target = self.target[index]

        # if self.transform is not None:
        #     img = self.transform(img)

        # if self.target_transform is not None:
        #     target = self.target_transform(target)

        # return img, target

    def __len__(self):
        return len(self.image_paths)

In [0]:
#del depth_dataset, depth_dataloader

In [0]:
depth_dataset = DenseDepth(root='./', transform=T.Compose([T.ToTensor()]))

In [0]:
depth_dataloader = DataLoader(depth_dataset, batch_size=4, shuffle=True, num_workers=1)

In [0]:
def get_mean_and_std(dataset):
    '''Compute the mean and std value of dataset.'''
    dataloader = torch.utils.data.DataLoader(
        dataset, batch_size=1, shuffle=True, num_workers=1)
    mean = torch.zeros(3)
    std = torch.zeros(3)
    print('==> Computing mean and std..')
    for images in tqdm(dataloader):
        for i in range(3):
            mean[i] += images[:, i, :, :].mean()
            std[i] += images[:, i, :, :].std()
    mean.div_(len(dataset))
    std.div_(len(dataset))
    return mean, std

## Compute Mean and Std dev. for BG images

In [0]:
bg_dataset = DenseDepth(root='./', data='bg', transform=T.Compose([T.ToTensor()]))
mean, std = get_mean_and_std(bg_dataset)

==> Computing mean and std..


In [0]:
[f'{m:.15f}' for m in mean], [f'{s:.15f}' for s in std]

(['0.514309406280518', '0.506399869918823', '0.491128325462341'],
 ['0.222627013921738', '0.222440108656883', '0.236961439251900'])

## Compute Mean and Std dev. for fg_bg images

In [0]:
fg_bg_dataset = DenseDepth(root='./', data='fg_bg', transform=T.Compose([T.ToTensor()]))
mean, std = get_mean_and_std(fg_bg_dataset)

==> Computing mean and std..


In [0]:
[f'{m:.15f}' for m in mean], [f'{s:.15f}' for s in std]

(['0.517005503177643', '0.505432009696960', '0.487791448831558'],
 ['0.229048311710358', '0.227772325277328', '0.242317050695419'])

## Compute Mean and Std dev. for fg_bg_mask

In [0]:
fg_bg_mask_dataset = DenseDepth(root='./', data='fg_bg_mask', transform=T.Compose([T.ToTensor()]))
mean, std = get_mean_and_std(fg_bg_mask_dataset)

==> Computing mean and std..


In [0]:
[f'{m:.15f}' for m in mean], [f'{s:.15f}' for s in std]

(['0.051495332270861', '0.051495332270861', '0.051495332270861'],
 ['0.214586526155472', '0.214586526155472', '0.214586526155472'])

## Compute Mean and Std dev. for depth_fg_bg

In [10]:
depth_fg_bg_dataset = DenseDepth(root='./', data='depth_fg_bg', transform=T.Compose([T.ToTensor()]))
mean, std = get_mean_and_std(depth_fg_bg_dataset)

==> Computing mean and std..


In [11]:
[f'{m:.15f}' for m in mean], [f'{s:.15f}' for s in std]

(['0.374479949474335', '0.374479949474335', '0.374479949474335'],
 ['0.210883036255836', '0.210883036255836', '0.210883036255836'])

In [0]:
f = open(root_path, 'rb')
self.zip_content = f.read()
f.close()
self.zip_file = zipfile.ZipFile(io.BytesIO(self.zip_content), 'r')